# Note
This Notebook contains all three parts of the assignment and is therefore submitted three times. 


## Part1

In [1]:
import pandas as pd

In [2]:
#Load Data
df0 = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df0.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
# Drop rows with unassigned borough
df1 = df0[df0.Borough != 'Not assigned']
df1.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
# Test if there are duplicate postal codes in the data
import numpy as np
unique_pc = np.unique(df1[['Postal Code']])
print("(True or False) Are all Postal Codes unique in the data?: ",len(unique_pc) == df1.shape[0])

(True or False) Are all Postal Codes unique in the data?:  True


Comment: Our exercise stated that 'M5A' is listed twice in the dataset, but this seems to be outdated. Our dataset is already in the shape needed for the exercise. 

In [5]:
# Merge neighborhoods, which are 'Not assigned' with their borough.
# Check if there exists 'Not assigned' neighborhoods

notAssignedNeigh = False
for d in df1['Neighbourhood']== 'Not assigned':
    if(d == True):
        notAssignedNeigh = True

if (notAssignedNeigh):
    print("There are neighborhoods, which are 'Not assigned'.")
else:
    print("There are no 'Not assigned' neighborhoods.")
    

There are no 'Not assigned' neighborhoods.


Comment: Checked the wikipedia page to be certain. Every 'Not assigned' Neighborhood has a 'Not assigned' Borough. The dataframe already fits our needs.

In [6]:
clean_df = df1
clean_df.shape

(103, 3)

## Part 2

First we will get all coordinates.

In [7]:
# install and import geocoder
!pip install geocoder 
import geocoder # import geocoder

Could not work it out how to get the coordinates. Tried everything, even as simple as:

In [8]:
lat_lng_coords = None
i = 0
while (lat_lng_coords is None):
    g = geocoder.google('M5G, Toronto, Ontario')
    lat_lng_coords = g.latlng
    print(lat_lng_coords)
    i += 1
    if(i>=50):
        break

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


Tried it online and 'M5G, Toronto, Ontario' is a valid query.

Hence, load alternative csv.

In [9]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
# merge dataframes using 'Postal Code' as key
df = pd.merge(clean_df,df_geo_coo, on="Postal Code")
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Part 3

We will only be interested in the 'Toronto' Borough. Therefore, we will create a new dataframe df_toronto.

In [11]:
df_toronto = df[df['Borough'].str.contains("Toronto")]
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


Now lets visualize the data, as learned...

In [12]:
#install and import folium
!pip install folium
import folium

In [13]:
#Create map of Toronto

latitude_toronto = np.mean(df_toronto['Latitude'])
longitude_toronto = np.mean(df_toronto['Longitude'])
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Even though the map is present in jupyter notebook, it seems to be missing on github.
So I provided a picture here:

Not lets cluster the locations by its local data:

In [14]:
#import necessary tools
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

Create Training set X which contains the coordinates of our dataframe.
Then scale it.

In [15]:
X = df_toronto.values[:,3:]
#X = nan_to_num(X) #As there are not NaN entries, we will not need this line here.
cluster_dataset = StandardScaler().fit_transform(X)

Run k means.

In [16]:
num_clusters = 3

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

In [17]:
df_toronto["Labels"] = labels
df_toronto.head(20)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Labels
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1


Visualize clustered data:

In [18]:
#Create map of Toronto

latitude_toronto = np.mean(df_toronto['Latitude'])
longitude_toronto = np.mean(df_toronto['Longitude'])
colors = ['blue','red','green']
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# add markers to map
for labels, lat, lng, borough, neighborhood in zip(df_toronto['Labels'], df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=colors[labels],
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto